In [51]:
import pandas as pd
import numpy as np
import QuantLib as ql
import matplotlib.pyplot as plt

%config InlineBackend.figure_format='retina'

# Swaption pricing
Below we illustrate 4 types of swaption pricing:
- using a scalar implied vol,
- using a flat volatility surface,
- using (non-flat) ATM volatility surface,
- using full (expiry x tenor x strike) volcube.

This is shown in terms of both (shifted) lognormal and normal model scopes.

# Initial setup

In [194]:
today = ql.Date(7, 10, 2022)
ql.Settings.instance().evaluationDate = today


forecast_curve = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(0.05)), ql.Actual360()))
discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(0.05)), ql.Actual360()))

# Definition of a swaption

In [365]:
calendar     = ql.TARGET()
swap         = ql.MakeVanillaSwap(ql.Period('5y'), ql.Euribor6M(forecast_curve), 0.045, ql.Period('5y'), discountingTermStructure = discount_curve) # payer swap
exerciseDate = calendar.advance(today, ql.Period('5y'))
swaption     = ql.Swaption(swap, ql.EuropeanExercise(exerciseDate))

# Various volatility structures

## Scalar implied volatility

### (Shifted) lognormal model
When we use shifted lognormal model with a scalar volatility, we define the shifted lognormal property in the pricing engine: `ql.BlackSwaptionEngine`, which has `shift` as its 4th parameter.

In [366]:
shift = 0.02
volatitility = ql.QuoteHandle(ql.SimpleQuote(0.2))

swaption.setPricingEngine(ql.BlackSwaptionEngine(discount_curve, volatitility, ql.SimpleDayCounter(), shift))
npv = swaption.NPV()

print(f'Swaption price in shifted lognormal model (shift={shift:.2f}) is {npv:.8f}')

Swaption price in shifted lognormal model (shift=0.02) is 0.05332196


### Bachelier (normal) model

In [367]:
volatitility = ql.QuoteHandle(ql.SimpleQuote(0.013577))

swaption.setPricingEngine(ql.BachelierSwaptionEngine(discount_curve, volatitility))
npv = swaption.NPV()

print(f'Swaption price in Bachelier (normal) model is {npv:.8f}')

Swaption price in Bachelier (normal) model is 0.05332226


## Flat volsurface
For a flat volsurfraced-based valuation, the key object is `ql.ConstantSwaptionVolatility` which defines the flat volsurface.  
The model choice must be defined both in:
- `ql.ConstantSwaptionVolatility` object (the type of a volatility + potentially a shift), and as
- pricing engine (`ql.BlackSwaptionEngine`, or `ql.BachelierSwaptionEngine`).

Note that when working with volsurface(s), its handle (`ql.SwaptionVolatilityStructureHandle`) needs to be used in the pricing engine.

### (Shifted) lognormal model
The shift is only part of flat volsurface `ql.ConstantSwaptionVolatility`. Note that this is different from the scalar case, where the shift was part of the engine. In the flat-volsurface case the engine inherits the shfit from the flat volsurface shift.

In [317]:
model = ql.ShiftedLognormal
shift = 0.02
volatility = ql.QuoteHandle(ql.SimpleQuote(0.3))

constantSwaptionVol = ql.ConstantSwaptionVolatility(0, ql.TARGET(), ql.ModifiedFollowing, volatility, ql.SimpleDayCounter(), model, shift)
constantSwaptionVol = ql.SwaptionVolatilityStructureHandle(constantSwaptionVol)

swaption.setPricingEngine(ql.BlackSwaptionEngine(discount_curve, constantSwaptionVol))
npv = swaption.NPV()

print(f'Swaption price in shifted lognormal model (shift={shift:.2f}) is {npv:.8f}')

Swaption price in shifted lognormal model (shift=0.02) is 0.05420326


### Bachelier (normal) model

In [259]:
model = ql.Normal
volatility = ql.QuoteHandle(ql.SimpleQuote(0.013577))

constantSwaptionVol = ql.ConstantSwaptionVolatility(0, ql.TARGET(), ql.ModifiedFollowing, volatility, ql.SimpleDayCounter(), model)
constantSwaptionVol = ql.SwaptionVolatilityStructureHandle(constantSwaptionVol)

swaption.setPricingEngine(ql.BachelierSwaptionEngine(discount_curve, constantSwaptionVol))
npv = swaption.NPV()

print(f'Swaption price in Bachelier (normal) model is {npv:.8f}')

Swaption price in Bachelier (normal) model is 0.05331147


## Volsurface (expiry X tenor)
In this case volatilitity surface is not flat and represented by a (expiry x tenor) matrix. The 'strike' dimension doesn't exist.

- key volsurface object is `ql.SwaptionVolatilityMatrix` which contains the matrix of (expiry x tenor) quotes plus information about model used.

### (Shifted) lognormal model
When shifted lognormal model is to be used, the shifted lognormal property needs to be specified in `ql.SwaptionVolatilityMatrix` (and pricing engine, as always).  
Matrix of shifts also needs to be passed into `ql.SwaptionVolatilityMatrix`. If not passed, the model will assume zero shift, i.e. a pure lognormal model.

In [369]:
expiries  = ['1M', '2M', '3M', '6M', '9M', '1Y', '18M', '2Y', '3Y', '4Y', '5Y', '7Y','10Y', '15Y', '20Y', '25Y', '30Y']
tenors    = ['1Y', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y', '15Y', '20Y', '25Y', '30Y']

# volsurface in QuantLib is understood as (expiry x tenor) surface
vols = [
    [8.6, 12.8, 19.5, 26.9, 32.7, 36.1, 38.7, 40.9, 42.7, 44.3, 48.8, 50.4, 50.8, 50.4],
    [9.2, 13.4, 19.7, 26.4, 31.9, 35.2, 38.3, 40.2, 41.9, 43.1, 47.8, 49.9, 50.7, 50.3],
    [11.2, 15.3, 21.0, 27.6, 32.7, 35.3, 38.4, 40.8, 42.6, 44.5, 48.6, 50.5, 50.9, 51.0],
    [12.9, 17.1, 22.6, 28.8, 33.5, 36.0, 38.8, 41.0, 43.0, 44.6, 48.7, 50.6, 51.1, 51.0],
    [14.6, 18.7, 24.6, 30.1, 34.2, 36.9, 39.3, 41.3, 43.2, 44.9, 48.9, 51.0, 51.3, 51.5],
    [16.5, 20.9, 26.3, 31.3, 35.0, 37.6, 40.0, 42.0, 43.7, 45.3, 48.8, 50.9, 51.4, 51.7],
    [20.9, 25.3, 30.0, 34.0, 37.0, 39.5, 41.9, 43.4, 45.0, 46.4, 49.3, 51.0, 51.3, 51.9],
    [25.1, 28.9, 33.2, 36.2, 39.2, 41.2, 43.2, 44.7, 46.0, 47.3, 49.6, 51.0, 51.3, 51.6],
    [34.0, 36.6, 39.2, 41.1, 43.2, 44.5, 46.1, 47.2, 48.0, 49.0, 50.3, 51.3, 51.3, 51.2],
    [40.3, 41.8, 43.6, 44.9, 46.1, 47.1, 48.2, 49.2, 49.9, 50.5, 51.2, 51.3, 50.9, 50.7],
    [44.0, 44.8, 46.0, 47.1, 20, 49.1, 49.9, 50.7, 51.4, 51.9, 51.6, 51.4, 50.6, 50.2],
    [49.6, 49.7, 50.4, 51.2, 51.8, 52.2, 52.6, 52.9, 53.3, 53.8, 52.6, 51.7, 50.4, 49.6],
    [53.9, 53.7, 54.0, 54.2, 54.4, 54.5, 54.5, 54.4, 54.4, 54.9, 53.1, 51.8, 50.1, 49.1],
    [54.0, 53.7, 53.8, 53.7, 53.5, 53.6, 53.5, 53.3, 53.5, 53.7, 51.4, 49.8, 47.9, 46.6],
    [52.8, 52.4, 52.6, 52.3, 52.2, 52.3, 52.0, 51.9, 51.8, 51.8, 49.5, 47.4, 45.4, 43.8],
    [51.4, 51.2, 51.3, 51.0, 50.8, 50.7, 50.3, 49.9, 49.8, 49.7, 47.6, 45.3, 43.1, 41.4],
    [49.6, 49.6, 49.7, 49.5, 49.5, 49.2, 48.6, 47.9, 47.4, 47.1, 45.1, 42.9, 40.8, 39.2]
]

# build 'shifts' matrix with the same dimension as the volsurface; this represents a shift used for each quote in the volsurface. We use the same shift for each quote
shifts      = ql.Matrix(np.full_like(vols, 0.02).tolist())

expiries    = [ql.Period(x) for x in expiries]
tenors      = [ql.Period(x) for x in tenors]
vols        = ql.Matrix((np.array(vols)/100).tolist()) # volsurface needs to be a matrix

volsurface = ql.SwaptionVolatilityMatrix(calendar, ql.ModifiedFollowing, expiries, tenors, vols, ql.SimpleDayCounter(), False, ql.ShiftedLognormal, shifts)
volsurface = ql.SwaptionVolatilityStructureHandle(volsurface)

swaption.setPricingEngine(ql.BlackSwaptionEngine(discount_curve, volsurface))
swaption.NPV()

volsurface_ATM_lognormal = volsurface # store the vols as ATM vols for use in other example

print(f'Swaption price in shifted lognormal model is {npv:.8f}. Shift used has been taken from the shift matrix.')

Swaption price in shifted lognormal model is 0.05332226. Shift used has been taken from the shift matrix.


### Bachelier (normal) model
The inputs are simpler than in the lognormal model above because shift doesn't need to be passed. We only need to specify a `ql.Normal` model in `ql.SwaptionVolatilityMatrix` and use `ql.BachelierSwaptionEngine` for pricing.

In [371]:
expiries  = ['1M', '2M', '3M', '6M', '9M', '1Y', '18M', '2Y', '3Y', '4Y', '5Y', '7Y','10Y', '15Y', '20Y', '25Y', '30Y']
tenors    = ['1Y', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y', '15Y', '20Y', '25Y', '30Y']

# volsurface in QuantLib is understood as (expiry x tenor) surface
vols = [
    [8.6, 12.8, 19.5, 26.9, 32.7, 36.1, 38.7, 40.9, 42.7, 44.3, 48.8, 50.4, 50.8, 50.4],
    [9.2, 13.4, 19.7, 26.4, 31.9, 35.2, 38.3, 40.2, 41.9, 43.1, 47.8, 49.9, 50.7, 50.3],
    [11.2, 15.3, 21.0, 27.6, 32.7, 35.3, 38.4, 40.8, 42.6, 44.5, 48.6, 50.5, 50.9, 51.0],
    [12.9, 17.1, 22.6, 28.8, 33.5, 36.0, 38.8, 41.0, 43.0, 44.6, 48.7, 50.6, 51.1, 51.0],
    [14.6, 18.7, 24.6, 30.1, 34.2, 36.9, 39.3, 41.3, 43.2, 44.9, 48.9, 51.0, 51.3, 51.5],
    [16.5, 20.9, 26.3, 31.3, 35.0, 37.6, 40.0, 42.0, 43.7, 45.3, 48.8, 50.9, 51.4, 51.7],
    [20.9, 25.3, 30.0, 34.0, 37.0, 39.5, 41.9, 43.4, 45.0, 46.4, 49.3, 51.0, 51.3, 51.9],
    [25.1, 28.9, 33.2, 36.2, 39.2, 41.2, 43.2, 44.7, 46.0, 47.3, 49.6, 51.0, 51.3, 51.6],
    [34.0, 36.6, 39.2, 41.1, 43.2, 44.5, 46.1, 47.2, 48.0, 49.0, 50.3, 51.3, 51.3, 51.2],
    [40.3, 41.8, 43.6, 44.9, 46.1, 47.1, 48.2, 49.2, 49.9, 50.5, 51.2, 51.3, 50.9, 50.7],
    [44.0, 44.8, 46.0, 47.1, 135.77, 49.1, 49.9, 50.7, 51.4, 51.9, 51.6, 51.4, 50.6, 50.2],
    [49.6, 49.7, 50.4, 51.2, 51.8, 52.2, 52.6, 52.9, 53.3, 53.8, 52.6, 51.7, 50.4, 49.6],
    [53.9, 53.7, 54.0, 54.2, 54.4, 54.5, 54.5, 54.4, 54.4, 54.9, 53.1, 51.8, 50.1, 49.1],
    [54.0, 53.7, 53.8, 53.7, 53.5, 53.6, 53.5, 53.3, 53.5, 53.7, 51.4, 49.8, 47.9, 46.6],
    [52.8, 52.4, 52.6, 52.3, 52.2, 52.3, 52.0, 51.9, 51.8, 51.8, 49.5, 47.4, 45.4, 43.8],
    [51.4, 51.2, 51.3, 51.0, 50.8, 50.7, 50.3, 49.9, 49.8, 49.7, 47.6, 45.3, 43.1, 41.4],
    [49.6, 49.6, 49.7, 49.5, 49.5, 49.2, 48.6, 47.9, 47.4, 47.1, 45.1, 42.9, 40.8, 39.2]
]

expiries    = [ql.Period(x) for x in expiries]
tenors      = [ql.Period(x) for x in tenors]
vols        = ql.Matrix((np.array(vols)/10000).tolist()) # conver volsurface to a bp vol matrix

volsurface = ql.SwaptionVolatilityMatrix(calendar, ql.ModifiedFollowing, expiries, tenors, vols, ql.SimpleDayCounter(), False, ql.Normal)
volsurface = ql.SwaptionVolatilityStructureHandle(volsurface)

swaption.setPricingEngine(ql.BachelierSwaptionEngine(discount_curve, volsurface))
swaption.NPV()

volsurface_ATM_normal = volsurface # store the vols as ATM vols for use in other example

print(f'Swaption price in Bachelier (normal) model is {npv:.8f}')

Swaption price in Bachelier (normal) model is 0.05332226


## Volcube (expiry x tenor x strike)
Volatility cube is the most complex volatility object for swaption pricing. Volatilities are linearly interpolated. The input is ATM vol matrix and then set of offsets that represent the volcube, taken relative to the ATM quotes.

The type of vols used are inferred from the input ATM vol matrix (which has this info embedded).

The key object is `ql.SwaptionVolCube2` which consumes:
- matrix of ATM vols (`ql.SwaptionVolatilityMatrix`) wrapped in a `ql.SwaptionVolatilityStructureHandle` handle,
- `strikeSpreads` : strike offsets from ATM for which we supply volatolities
- `volSpreads`: volatility offsets form ATM vols. volATM+`volSpreads` give the desired volatility quote. 
- `swapIndexBase`: short swap index, needs both forecasting and discounting curve. This is to 'split' the volsurface into short tenors, and long tenors part
- `shortSwapIndexBase`: swap index, needs both forecasting and discounting curve. This is to 'split' the volsurface into short tenors and long tenors part

The 2nd and 3rd bullet point define how we represent the quotes. Clearly, the quotes are defined as 'offsets'. Let's do an example of how this offset works:
Let's assume: $K_{\text{ATM}} = \text{ATM} = 0.05$ and $K_{\text{ATM+1%}} = \text{ATM} + 0.01 = 0.06$. Now consider the associated volatilities are $\sigma_{K_{\text{ATM}}} = 0.20$ and $\sigma_{K_{\text{ATM+1%}}} = 0.25$. To add this non-ATM quote in the volcube, we would use (strikeSpread = 0.01, volSpread = 0.05) because the desired non-ATM strike is 1% offset from ATM, and the associated vol is 0.05 offset from the ATM vol.


On the last two points, we recommend reading https://quant.stackexchange.com/questions/57639/quantlib-swaption-vol-cube . There, it is mentioned that for the very short swap tenors (usually <1Y), the underlying swap pays on the fixed leg typically more frequently (than annually), and for swap > shortSwapIndexBase (usuall >= 1Y) the swap pays less frequently, e.g. 1Y. We can thus fully specify this behaviour. The swap index definitions are according to https://quantlib-python-docs.readthedocs.io/en/latest/indexes.html?highlight=EuriborSwapIsdaFixA .

The `volSpreads` containing the quotes ('offsets' from ATM vols) object is a 2d array that must be passed in as a list of lists (_not_ a Matrix!) of simple quote handles. Note that it is 2d array storing a 3d cube (expiry x tenor x strike). So the two first two dimensions are flattened (i.e. `volSpreads` is expirytenor x strike 2d array).
It means that `volSpreads` stores the quotes as follows:  
[[0.01, 0, 0.015], -> this is for (expiry1, tenor**1**, {3 strikes})  
 [0.02, 0, 0.025]] -> this is for (expiry1, tenor**2**, {3 strikes})  
 ...
 
 Useful reference is also https://programtalk.com/vs2/?source=python/7309/QuantLib-SWIG/Python/test/cms.py

### (Shifted) lognormal model

In [373]:
expiries =  ['1y', '2y', '5y']
tenors   =  ['5Y', '10Y']
strikeSpreads = [ -0.01, 0, 0.01]
volSpreads = [
    [0.01, 0, 0.02],   # option 1Y, swap 5y
    [0.03, 0, 0.03],   # option 1Y, swap 10y
    [0.04, 0, 0.06],   # option 2Y, swap 5y
    [0.07, 0, 0.08],   # option 2Y, swap 10y
    [0.09, 0, 0.10],   # option 5Y, swap 5y
    [0.11, 0, 0.12],   # option 5Y, swap 10y
]

expiries    = [ql.Period(x) for x in expiries]
tenors      = [ql.Period(x) for x in tenors]
volSpreads  = pd.DataFrame(volSpreads).applymap(lambda x: ql.QuoteHandle(ql.SimpleQuote(x))).values.tolist() # list of list of simple vol quotes

# this defines the underlying swap in the surface, as per: https://quant.stackexchange.com/questions/57639/quantlib-swaption-vol-cube 
swapIndexBase      = ql.EuriborSwapIsdaFixA(ql.Period(1, ql.Years), forecast_curve, discount_curve)
shortSwapIndexBase = ql.EuriborSwapIsdaFixA(ql.Period(1, ql.Years), forecast_curve, discount_curve)
vegaWeightedSmileFit = False

volCube = ql.SwaptionVolCube2(volsurface_ATM_lognormal,
                              expiries,
                              tenors,
                              strikeSpreads,
                              volSpreads,
                              swapIndexBase,
                              shortSwapIndexBase,
                              vegaWeightedSmileFit)
volCube = ql.SwaptionVolatilityStructureHandle(volCube)
volCube.disableExtrapolation()


swaption.setPricingEngine(ql.BlackSwaptionEngine(discount_curve, volCube))
npv = swaption.NPV()

print(f'Swaption price in shifted lognormal model is {npv:.8f}. Shift used has been taken from the shift matrix.')

Swaption price in shifted lognormal model is 0.06559349. Shift used has been taken from the shift matrix.


### Bachelier (normal) model

In [372]:
expiries =  ['1y', '2y', '5y']
tenors   =  ['5Y', '10Y']
strikeSpreads = [ -0.01, 0, 0.01]
volSpreads = [
    [0.001, 0, 0.002],   # option 1Y, swap 5y
    [0.003, 0, 0.003],   # option 1Y, swap 10y
    [0.004, 0, 0.006],   # option 2Y, swap 5y
    [0.007, 0, 0.008],   # option 2Y, swap 10y
    [0.009, 0, 0.010],   # option 5Y, swap 5y
    [0.011, 0, 0.012],   # option 5Y, swap 10y
]

expiries    = [ql.Period(x) for x in expiries]
tenors      = [ql.Period(x) for x in tenors]
volSpreads  = pd.DataFrame(volSpreads).applymap(lambda x: ql.QuoteHandle(ql.SimpleQuote(x))).values.tolist() # list of list of simple vol quotes

# this defines the underlying swap in the surface, as per: https://quant.stackexchange.com/questions/57639/quantlib-swaption-vol-cube 
swapIndexBase      = ql.EuriborSwapIsdaFixA(ql.Period(1, ql.Years), forecast_curve, discount_curve)
shortSwapIndexBase = ql.EuriborSwapIsdaFixA(ql.Period(1, ql.Years), forecast_curve, discount_curve)
vegaWeightedSmileFit = False

volCube = ql.SwaptionVolCube2(volsurface_ATM_normal,
                              expiries,
                              tenors,
                              strikeSpreads,
                              volSpreads,
                              swapIndexBase,
                              shortSwapIndexBase,
                              vegaWeightedSmileFit)
volCube = ql.SwaptionVolatilityStructureHandle(volCube)
volCube.disableExtrapolation()


swaption.setPricingEngine(ql.BachelierSwaptionEngine(discount_curve, volCube))
npv = swaption.NPV()

print(f'Swaption price in Bachelier (normal) model is {npv:.8f}')

0.07189081869138095